# Levantamiento de la base de datos de Votos

In [18]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

 Usaremos la base de datos de los  [Cómputos Distritales para la Elección Presidencial de 2018 a nivel casilla](https://computos2018.ine.mx/#/descargaBase) del Instituto Nacional Electoral (INE) de México para analizar los votos obtenidos por los candidatos presidenciales en las elecciones de 2018.

In [2]:
# El data frame tiene lineas de encabezado adicionales que se deben omitir
Votos = pd.read_csv("Bases_Votos/presidencia.csv", sep="|", 
                    encoding="latin1",engine="python",skiprows=6)
Votos.head()

,CLAVE_CASILLA,CLAVE_ACTA,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO,NOMBRE_DISTRITO,SECCION,ID_CASILLA,TIPO_CASILLA,EXT_CONTIGUA,CASILLA,NUM_ACTA_IMPRESO,PAN,PRI,PRD,PVEM,PT,MOVIMIENTO CIUDADANO,NUEVA ALIANZA,MORENA,ENCUENTRO SOCIAL,PAN_PRD_MC,PAN_PRD,PAN_MC,PRD_MC,PRI_PVEM_NA,PRI_PVEM,PRI_NA,PVEM_NA,PT_MORENA_PES,PT_MORENA,PT_PES,MORENA_PES,CAND_IND_01,CAND_IND_02,CNR,VN,TOTAL_VOTOS_CALCULADOS,LISTA_NOMINAL_CASILLA,OBSERVACIONES,MECANISMOS_TRASLADO,FECHA_HORA,Unnamed: 42,Unnamed: 43
0,'010000M0100',010000M010001,1,AGUASCALIENTES,0,,0,1,M,0,,1,247,37,8,1,24,4,0,378,5,27,1,2,0,6,0,1,0,47,7,1,1,0,26,5,9,837,1683,-,-,05/07/18 14:00:00.000000,NaN,NaN
1,'010338B0100',010338B010002,1,AGUASCALIENTES,1,JESÚS MARÍA,338,1,B,0,Rural,2,62,43,2,1,21,4,3,153,5,1,0,0,1,0,0,0,0,5,1,0,1,0,14,0,20,337,587,Cotejo (Levantada en Casilla),F,04/07/18 09:23:10.081000,NaN,NaN
2,'010338C0100',010338C010002,1,AGUASCALIENTES,1,JESÚS MARÍA,338,1,C,0,Rural,2,81,36,1,3,11,3,3,159,9,1,0,0,0,1,0,0,0,1,2,0,0,0,18,0,6,335,586,Cotejo (Levantada en Casilla),F,04/07/18 09:26:33.092000,NaN,NaN
3,'010338C0200',010338C020002,1,AGUASCALIENTES,1,JESÚS MARÍA,338,2,C,0,Rural,2,73,28,2,3,19,4,5,136,4,2,1,0,0,0,0,1,0,5,0,0,2,0,13,0,7,305,586,Cotejo (Levantada en Casilla),F,04/07/18 09:31:31.102000,NaN,NaN
4,'010339B0100',010339B010002,1,AGUASCALIENTES,1,JESÚS MARÍA,339,1,B,0,Rural,2,75,43,2,5,3,11,1,151,8,3,1,1,0,0,1,0,0,0,3,0,0,0,14,0,6,328,552,Cotejo (Levantada en Casilla),F-D,04/07/18 09:34:53.187000,NaN,NaN


La base Votos no tiene la columna municipios, por lo que nos ayudaremos de la base de LISTADO_CASILLAS_2018.csv para agregar esa información. a la base Votos.

In [3]:
#BASE DE DATOS CON LOS NOMBRES DE LOS ESTADOS SIN CARACTERES EXTRAÑOS
estados = pd. read_csv("Bases_Votos/EDos.csv", sep=",")
Listado_casillas = pd.read_csv("Bases_Votos/LISTADO_CASILLAS_2018.csv", sep=",", 
                              encoding="latin1",engine="python")
estados.head()

,ID_ESTADO,NOMBRE_ESTADO
0,1,AGUASCALIENTES
1,2,BAJA CALIFORNIA
2,3,BAJA CALIFORNIA SUR
3,4,CAMPECHE
4,5,COAHUILA


In [4]:
Listado_casillas_2= pd.merge(Listado_casillas, estados, on = "ID_ESTADO", how = "left")
# crear funcion que creee un Id unico por casilla para unir la base de datos 
def crear_id_casilla(df):
    df["ID"] = ("A" + df["ID_ESTADO"].astype(str).str.zfill(2) +
                              df["SECCION"].astype(str).str.zfill(4) +
                              df["TIPO_CASILLA"] + 
                              df["ID_CASILLA"].astype(str).str.zfill(2))
    return df

Listado_casillas_2 = crear_id_casilla(Listado_casillas_2)
Votos = crear_id_casilla(Votos)
Votos.columns

Index(['CLAVE_CASILLA', 'CLAVE_ACTA', 'ID_ESTADO', 'NOMBRE_ESTADO',
       'ID_DISTRITO', 'NOMBRE_DISTRITO', 'SECCION', 'ID_CASILLA',
       'TIPO_CASILLA', 'EXT_CONTIGUA', 'CASILLA', 'NUM_ACTA_IMPRESO', 'PAN',
       'PRI', 'PRD', 'PVEM', 'PT', 'MOVIMIENTO CIUDADANO', 'NUEVA ALIANZA',
       'MORENA', 'ENCUENTRO SOCIAL', 'PAN_PRD_MC', 'PAN_PRD', 'PAN_MC',
       'PRD_MC', 'PRI_PVEM_NA', 'PRI_PVEM', 'PRI_NA', 'PVEM_NA',
       'PT_MORENA_PES', 'PT_MORENA', 'PT_PES', 'MORENA_PES', 'CAND_IND_01',
       'CAND_IND_02', 'CNR', 'VN', 'TOTAL_VOTOS_CALCULADOS',
       'LISTA_NOMINAL_CASILLA', 'OBSERVACIONES', 'MECANISMOS_TRASLADO',
       'FECHA_HORA', 'Unnamed: 42', 'Unnamed: 43', 'ID'],
      dtype='object')

In [5]:
# Ahora voy a hacer una union de las dos bases de datos
Votos_completo = pd.merge(Votos, Listado_casillas_2[["ID", "NOMBRE_ESTADO", "ID_MUNICIPIO"]], 
                          on = "ID", how = "left")

filas_na = Votos_completo["NOMBRE_ESTADO_y"].isna().sum()
print("Renglones con NA:", filas_na )
print("Porcentaje renglones con NA:", filas_na / Votos_completo.shape[0] * 100)

Renglones con NA: 37
Porcentaje renglones con NA: 0.02129337085570576


No se realiza bien la unión entre las dos bases de datos, pero el porcentaje de  renglones con NA es minimo. Por tanto omitiremos estos renglones.

In [6]:
# Vamos a filtrar el conteo en casa casila junto con algunos datos de identificacion  y quitar los renglones 
# que contienen NA
Votos_completo = Votos_completo.dropna(subset=["NOMBRE_ESTADO_y"])
Votos_completo = Votos_completo.loc[:,
[
   "ID",                     "ID_ESTADO",              "NOMBRE_ESTADO_y",        "ID_MUNICIPIO",          
   "PAN",                    "PRI",                    "PRD",                    "PVEM",                  
   "PT",                     "MOVIMIENTO CIUDADANO",   "NUEVA ALIANZA",          "MORENA",                
   "ENCUENTRO SOCIAL",       "PAN_PRD_MC",             "PAN_PRD",                "PAN_MC",                
   "PRD_MC",                 "PRI_PVEM_NA",            "PRI_PVEM",               "PRI_NA",                
   "PVEM_NA",                "PT_MORENA_PES",          "PT_MORENA",              "PT_PES",                
   "MORENA_PES",             "CAND_IND_01",            "CAND_IND_02",            "CNR",                   
   "VN",                     "TOTAL_VOTOS_CALCULADOS", "LISTA_NOMINAL_CASILLA" ]]

Votos_completo.head()

,ID,ID_ESTADO,NOMBRE_ESTADO_y,ID_MUNICIPIO,PAN,PRI,PRD,PVEM,PT,MOVIMIENTO CIUDADANO,NUEVA ALIANZA,MORENA,ENCUENTRO SOCIAL,PAN_PRD_MC,PAN_PRD,PAN_MC,PRD_MC,PRI_PVEM_NA,PRI_PVEM,PRI_NA,PVEM_NA,PT_MORENA_PES,PT_MORENA,PT_PES,MORENA_PES,CAND_IND_01,CAND_IND_02,CNR,VN,TOTAL_VOTOS_CALCULADOS,LISTA_NOMINAL_CASILLA
1,A010338B01,1,AGUASCALIENTES,2.0,62,43,2,1,21,4,3,153,5,1,0,0,1,0,0,0,0,5,1,0,1,0,14,0,20,337,587
2,A010338C01,1,AGUASCALIENTES,2.0,81,36,1,3,11,3,3,159,9,1,0,0,0,1,0,0,0,1,2,0,0,0,18,0,6,335,586
3,A010338C02,1,AGUASCALIENTES,2.0,73,28,2,3,19,4,5,136,4,2,1,0,0,0,0,1,0,5,0,0,2,0,13,0,7,305,586
4,A010339B01,1,AGUASCALIENTES,2.0,75,43,2,5,3,11,1,151,8,3,1,1,0,0,1,0,0,0,3,0,0,0,14,0,6,328,552
5,A010339C01,1,AGUASCALIENTES,2.0,75,48,1,3,8,7,2,132,8,0,1,2,0,0,0,0,0,4,0,0,0,0,14,0,14,319,552


### Limpieza de los datos 

Verificamos que no haya valores faltantes en las columnas de interés.

In [7]:
Votos_completo.isna().sum()   

ID                        0
ID_ESTADO                 0
NOMBRE_ESTADO_y           0
ID_MUNICIPIO              0
PAN                       0
PRI                       0
PRD                       0
PVEM                      0
PT                        0
MOVIMIENTO CIUDADANO      0
NUEVA ALIANZA             0
MORENA                    0
ENCUENTRO SOCIAL          0
PAN_PRD_MC                0
PAN_PRD                   0
PAN_MC                    0
PRD_MC                    0
PRI_PVEM_NA               0
PRI_PVEM                  0
PRI_NA                    0
PVEM_NA                   0
PT_MORENA_PES             0
PT_MORENA                 0
PT_PES                    0
MORENA_PES                0
CAND_IND_01               0
CAND_IND_02               0
CNR                       0
VN                        0
TOTAL_VOTOS_CALCULADOS    0
LISTA_NOMINAL_CASILLA     0
dtype: int64

Ahora verificamos que en las columnas de los votos no hay valores tipo cadena 

In [8]:
columnas_de_votos = Votos_completo.columns[4:28]
Valores = {}
# vamos a ver si hay valores no numericos en las columnas 
for col in columnas_de_votos:
    serie_convertida = pd.to_numeric(Votos_completo[col], errors='coerce')
    filtro_problema = serie_convertida.isna() & Votos_completo[col].notna()
    # Obtiene los valores atípicos únicos de la columna
    valores_atipicos_col = Votos_completo.loc[filtro_problema, col].unique()

    if len(valores_atipicos_col) > 0:
        Valores[col] = {}
        Valores[col][valores_atipicos_col[0]] = len(filtro_problema)
        
Valores


{'PAN': {'-': 173726},
 'PRI': {'-': 173726},
 'PRD': {'-': 173726},
 'PVEM': {'-': 173726},
 'PT': {'-': 173726},
 'MOVIMIENTO CIUDADANO': {'-': 173726},
 'NUEVA ALIANZA': {'-': 173726},
 'MORENA': {'-': 173726},
 'ENCUENTRO SOCIAL': {'-': 173726},
 'PAN_PRD_MC': {'-': 173726},
 'PAN_PRD': {'-': 173726},
 'PAN_MC': {'-': 173726},
 'PRD_MC': {'-': 173726},
 'PRI_PVEM_NA': {'-': 173726},
 'PRI_PVEM': {'-': 173726},
 'PRI_NA': {'-': 173726},
 'PVEM_NA': {'-': 173726},
 'PT_MORENA_PES': {'-': 173726},
 'PT_MORENA': {'-': 173726},
 'PT_PES': {'-': 173726},
 'MORENA_PES': {'-': 173726},
 'CAND_IND_01': {'-': 173726},
 'CAND_IND_02': {'-': 173726},
 'CNR': {'-': 173726}}

Son solo valores faltantes en donde la base de datos no tiene información de los votos obtenidos en esa casilla. En este caso no se pueden sustituir los valores faltantes, ya que hacerlo seria modificar los resultados electorales. Por tanto, solo queda sustituirlos por 0 o por NA. 

In [9]:
for col in columnas_de_votos:
    Votos_completo[col] = pd.to_numeric(Votos_completo[col], errors='coerce')

Ahora lo que vamos a hacer es obtener el total de votos por municipio para cada candidato presidencial. Los votos se suman de la siguiente manera:

| Candidato | Coaliciones a Sumar |    
|:---|---:|
|Andres Manuel Lopez Obrador|PT, MORENA, ENCUENTRO SOCIAL, PT-MORENA-PES, PT-MORENA, PT-PES, MORENA-PES|  
|Ricardo Anaya Cortés|PAN, PRI, MOVIMIENTO CIUDADANO, PAN-PRD-MC, PAN-PRD, PAN-MC, PRD-MC|
 
```R

In [10]:
ColAMLO = [ "PT","MORENA" ,"ENCUENTRO SOCIAL","PT_MORENA_PES", "PT_MORENA","PT_PES","MORENA_PES"]
ColAnaya = ["PAN","PRD","MOVIMIENTO CIUDADANO","PAN_PRD_MC","PAN_PRD","PAN_MC","PRD_MC"]
Votos_completo["AMLO"] = Votos_completo[ColAMLO].sum(axis=1)
Votos_completo["Anaya"] = Votos_completo[ColAnaya].sum(axis=1)
Total_votos = Votos_completo["TOTAL_VOTOS_CALCULADOS"]
Porcentaje_AMLO = Votos_completo["AMLO"].sum() / Total_votos.sum() * 100
Porcentaje_Anaya = Votos_completo["Anaya"].sum() / Total_votos.sum() * 100
print(f"Porcentaje AMLO: {Porcentaje_AMLO:.2f}%")
print(f"Porcentaje Anaya: {Porcentaje_Anaya:.2f}%")

Porcentaje AMLO: 53.65%
Porcentaje Anaya: 22.05%


Los porcentajes no difieren mucho de los publicados oficialmente por el INE en [Cómputos Distritales 2018](https://computos2018.ine.mx/#/presidencia/nacional/1/1/1/1). Ahora voy a sacar los valores de la respuesta a nivel municipio, 

In [11]:
print(len(Votos_completo['ID_MUNICIPIO'].unique()))
print(len(Votos_completo['ID_MUNICIPIO']))

570
173726


En la columa 'ID_MUNICIPIO' hay claves que se repiten para cada estado, por lo que hay que crear una nueva columna que contenga la clave única de cada municipio y por estado. 

In [12]:
Votos_completo["ID_MUNICIPIO"] = Votos_completo["ID_MUNICIPIO"].astype(int)
Votos_completo["C_Municipio"] = (Votos_completo["ID_ESTADO"].astype(str).str.zfill(2) + "," +
                                Votos_completo["ID_MUNICIPIO"].astype(str).str.zfill(3))
y_1 = Votos_completo.groupby("C_Municipio")["AMLO"].sum()
y_2 = Votos_completo.groupby("C_Municipio")["Anaya"].sum()
y_3 = Votos_completo.groupby("C_Municipio")["TOTAL_VOTOS_CALCULADOS"].sum()

BD1 = pd.DataFrame({"AMLO": y_1, "Anaya": y_2, "Total_Votos": y_3}) 
BD1["AMLO"] = BD1["AMLO"].astype(int)
BD1["Anaya"] = BD1["Anaya"].astype(int)

Es mejor trabajar con porcentajes a nivel municipio, por lo que creamos las columnas correspondientes.

In [13]:
BD1["AMLO"] = (BD1["AMLO"]/BD1["Total_Votos"])*100
BD1["Anaya"] = (BD1["Anaya"]/BD1["Total_Votos"])*100
BD1 = BD1.reset_index()

La columna 'C_Municipio' nos servira de conexion con la base de datos del INEGI. Vinculamos esta clave con el nombre del municipio y el estado, las claves son obtenidas de la base de datos de [Catálogo de Claves de Entidades Federativas y Municipios](https://www.inegi.org.mx/app/ageeml/#).

In [14]:
Nombre_municipios = pd.read_csv("Bases_Votos/Nombres_municipios.CSV", encoding="UTF-8")
Nombre_municipios.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN
0,1,Aguascalientes,1,Aguascalientes
1,1,Aguascalientes,2,Asientos
2,1,Aguascalientes,3,Calvillo
3,1,Aguascalientes,4,Cosío
4,1,Aguascalientes,5,Jesús María


Vamos a crear el identificador como en BD1 como "Entidad,Municipio"

In [15]:
Nombre_municipios["C_Municipio"] = (Nombre_municipios["ENTIDAD"].astype(str).str.zfill(2) + "," +
                                    Nombre_municipios["MUN"].astype(str).str.zfill(3)                                       
                                    )
BD1 = pd.merge(BD1, Nombre_municipios[["ENTIDAD", "MUN", "C_Municipio"]], on="C_Municipio", how="left")

In [17]:
BD1.to_csv("Votos_por_Municipio.csv", index=False, encoding="utf-8")